This notebook will develope tools to calculate the pressure from the hydrostatic relation. 

In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

from nowcast import analyze
from salishsea_tools import viz_tools, psu_tools

import grid_time
import gsw

%matplotlib inline

Plan: 
* Use hydrostatic approximation to calculate pressure:
$ \frac{\partial p}{\partial z} = -\rho g $ 
with $p(z=0) = 0$
* For now, calculate density with the function Susan gave me. How should this change with TEOS-10?
* To enforce $p(z=0)=0$, I think it is best to interpolate rho onto w grid and then integrate for pressure with upper BC and then interpolate back to t grid 

# Loading

In [2]:
mesh_mask = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/mesh_mask_SalishSea2.nc')
grid = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')

bathy = grid.variables['Bathymetry'][:]
# Sample file for testing calculations
fT = nc.Dataset('/results/SalishSea/nowcast/01apr15/SalishSea_1h_20150401_20150401_grid_T.nc')

In [3]:
tmask = mesh_mask.variables['tmask'][:]
e3t_0 = mesh_mask.variables['e3t'][:]
gdept_0 = mesh_mask.variables['gdept'][:]
gdepw_0 = mesh_mask.variables['gdepw'][:]

In [4]:
ssh = fT.variables['sossheig'][:]
sal = fT.variables['vosaline'][:]
temp = fT.variables['votemper'][:]
rho = psu_tools.calculate_density(temp, sal)

In [5]:
e3t_t, e3w_t, gdept_t, gdepw_t = grid_time.calculate_vertical_grids(e3t_0[0,...], tmask[0,...], ssh)

In [6]:
def interp_depth_time_dependent(var, depth, depth_int):
    var_int = np.zeros(var.shape)
    for t in np.arange(var_int.shape[0]):
        for j in np.arange(var_int.shape[2]):
            for i in np.arange(var_int.shape[3]):
                var_int[t,:,j,i] = np.interp(depth_int[t,:,j,i], depth[t,:,j,i], var[t,:,j,i],right = np.nan)
    return var_int

In [7]:
def calculate_pressure(rho, e3w_t, gdept_t, gdepw_t, tmask):
    """Integrate density to calulate pressure.
    Use time-dependent scale factors"""
    g = 9.81 # is there a constant module that I an grab this from?
    
    # Integrate density to w-levels
    rho_m = np.ma.array(rho, mask = np.ones(rho.shape)-tmask) 
    gdept_m =  np.ma.array(gdept_t, mask = np.ones(rho.shape) - tmask)
    gdepw_m =  np.ma.array(gdepw_t, mask = np.ones(rho.shape) - tmask)
    rho_w = interp_depth_time_dependent(rho_m, gdept_m, gdepw_m)
    # integrate density to get pressure
    p = np.cumsum(rho_w*e3w_t*tmask, axis=1)*tmask
    p = p - np.expand_dims(p[:, 0, ...], axis=1)
    # interp back onto t grids
    p_t = interp_depth_time_dependent(p, gdepw_m, gdept_m)
    
    return p_t
    

In [8]:
p_t = calculate_pressure(rho, e3w_t, gdept_t, gdepw_t, tmask)

In [9]:
p_t[0,:,400,300]

array([   518.52009916,   1557.48304459,   2600.52908689,   3646.65245417,
         4693.84047739,   5740.95106667,   6787.44093108,   7833.11684329,
         8878.06680522,   9922.67076825,  10967.75076421,  12014.96189082,
        13067.58077898,  14132.02727517,  15220.77888209,  16357.955306  ,
        17589.96062042,  19005.25806778,  20769.28113765,  22530.93184911,
        11130.21872531,  -1036.08051487,  -1036.08051487,  -1036.08051487,
        -1036.08051487,  -1036.08051487,  -1036.08051487,  -1036.08051487,
        -1036.08051487,  -1036.08051487,  -1036.08051487,  -1036.08051487,
        -1036.08051487,  -1036.08051487,  -1036.08051487,  -1036.08051487,
        -1036.08051487,  -1036.08051487,  -1036.08051487,             nan])